In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Schizophrenia"
cohort = "GSE165813"

# Input paths
in_trait_dir = "../../input/GEO/Schizophrenia"
in_cohort_dir = "../../input/GEO/Schizophrenia/GSE165813"

# Output paths
out_data_file = "../../output/preprocess/Schizophrenia/GSE165813.csv"
out_gene_data_file = "../../output/preprocess/Schizophrenia/gene_data/GSE165813.csv"
out_clinical_data_file = "../../output/preprocess/Schizophrenia/clinical_data/GSE165813.csv"
json_path = "../../output/preprocess/Schizophrenia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Astroblastomas exhibit radial glia stem cell lineages and differential expression of imprinted and X-inactivation escape genes [Affymetrix]"
!Series_summary	"Astroblastoma (AB) is an unusual brain tumor of unknown origin. We performed an integrated clinicogenomic analysis of 36 AB-like tumors. Lesions with MN1-BEND2 fusions demonstrated decreased promoter methylation and increased expression of IGF2-H19 and DLK1-DIO3 imprinted region genes. They also relatively overexpressed genes highly expressed during fetal brain development prior to 25 post-conception weeks (pcw), including genes enriched in ventricular zone radial glia (vRG), and generally presented in young children. Other tumors highly expressed MAP kinase pathway, PI3K pathway and X-inactivation escape genes. These and a third group of tumors tended to occur in young adults and showed enriched expression of outer radial glia (oRG) and truncated radial glia (tRG) genes, and genes highly exp

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Examining gene expression data availability
# Based on the background information, this dataset contains Affymetrix data (microarray gene expression)
# The series title and summary mention gene expression analysis
is_gene_available = True

# Step 2: Variable availability and data type conversion

# 2.1 Data Availability
# Looking at the Sample Characteristics Dictionary:
# There's no explicit trait (Schizophrenia) data in the sample characteristics
# From the background information, this dataset is about astroblastomas and brain tumors, not directly related to schizophrenia
trait_row = None  # No schizophrenia trait data available

# No age information is available in the sample characteristics
age_row = None

# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since none of the variables are available, we'll define placeholder conversion functions
def convert_trait(value):
    if not value or pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Not applicable for this dataset as it doesn't have schizophrenia trait data
    return None

def convert_age(value):
    if not value or pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Not applicable as age data is not available
    return None

def convert_gender(value):
    if not value or pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Not applicable as gender data is not available
    return None

# Step 3: Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# Skip this step since trait_row is None (no clinical data available for schizophrenia)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Schizophrenia/GSE165813/GSE165813_series_matrix.txt.gz
Gene data shape: (23937, 48)
First 20 gene/probe identifiers:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2',
       'AADACL3', 'AADACL4', 'AADACP1', 'AADAT', 'AAGAB'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers, these appear to be standard human gene symbols
# For example: A1BG, A2M, AADAT, etc. are recognized human gene symbols
# No mapping should be required as these are already in the correct format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2-6. Process based on trait availability
# Since trait_row is None (determined in Step 2), we cannot create proper clinical data
# Skip clinical data processing and mark the dataset as not usable for Schizophrenia analysis

if trait_row is None:
    print("No Schizophrenia trait data available in this dataset. Skipping clinical data processing.")
    
    # Create an empty DataFrame as placeholder for linked_data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort information (with is_trait_available=False)
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,  # We confirmed trait data is not available
        is_biased=False,  # Providing a default value as required by the function
        df=linked_data,
        note="Dataset contains gene expression data but no Schizophrenia-related trait information."
    )
    
    print("Dataset is not usable for Schizophrenia analysis. No linked data file saved.")
else:
    # This branch won't execute since trait_row is None, but including it for completeness
    # Process clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
    
    # Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    
    # Check for bias in features
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # Validate and save cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=""
    )
    
    # Save the linked data if usable
    if is_usable and not linked_data.empty:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (23721, 48)


Normalized gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE165813.csv
No Schizophrenia trait data available in this dataset. Skipping clinical data processing.
Abnormality detected in the cohort: GSE165813. Preprocessing failed.
Dataset is not usable for Schizophrenia analysis. No linked data file saved.
